In [1]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)


In [2]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
    return tunique, tmatches, idx1, idx2



In [3]:
# Now loading in the catalog(s) from Tytler+2009
cheung2007 = (Table.read('Tables/Cheung_2007/table2.dat', readme = 'Tables/Cheung_2007/ReadMe', format='ascii.cds')).to_pandas()

cheung2007['System Type'] = "Recoil Candidate"
cheung2007['Name1'] = cheung2007['[C2007]']
cheung2007['Name2'] = cheung2007['[C2007]']
cheung2007['Selection Method'] = "X-ray Shaped Radio Source Candidate"
cheung2007['Confirmation Method'] = -99

cheung2007['z1'] = cheung2007['z']
cheung2007['z1_type'] = -100
cheung2007['z2'] = cheung2007['z']
cheung2007['z2_type'] = -100

# Creating an RA and Dec column for each source (and a duplicate column for each for the 'secondary' in each)
cheung2007['RA1'] = cheung2007['RAh'].astype(str) + ':' + cheung2007['RAm'].astype(str) + ':' + cheung2007['RAs'].astype(str)
cheung2007['Dec1'] = cheung2007['DE-'].astype(str) + cheung2007['DEd'].astype(str) + ':' + cheung2007['DEm'].astype(str) + ':' + cheung2007['DEs'].astype(str)

cheung2007['RA2'] = cheung2007['RA1']
cheung2007['Dec2'] = cheung2007['Dec1']

# And converting sexagesimal to degrees...
coordconvert = SkyCoord(ra = cheung2007['RA1'], dec = cheung2007['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
cheung2007['RA1_deg'] = coordconvert.ra.degree
cheung2007['Dec1_deg'] = coordconvert.dec.degree

cheung2007['RA2_deg'] = cheung2007['RA1_deg']
cheung2007['Dec2_deg'] = cheung2007['Dec1_deg']

# Dropping the original 7 columns for RA and Dec
cheung2007.drop(labels=['RAh','RAm','RAs','DE-','DEd','DEm','DEs'], axis=1, inplace=True)

# Dropping irrelevant columns that (as far as I know) we do not need...
cheung2007.drop(labels=['Seq','g-r','f_g-r','f_rmag','ID','S.365','S4.9','r_S4.9','alpha1','alpha2','OCat'], axis=1, inplace=True)
#'S1.4',

# Adding in columns for information about the coordinates
cheung2007['Equinox'] = "J2000"
cheung2007['Optical'] = 'Optical'
cheung2007['Coordinate_waveband'] = cheung2007['Optical'].where(((cheung2007['r_rmag']=='SDSS') | (cheung2007['r_rmag']=='APM') | (cheung2007['r_rmag']=='D89') | (cheung2007['r_rmag']=='USNO')), other='Radio')
cheung2007['Coordinate_Source'] = cheung2007['r_rmag'] # I believe this should be Cheung's column 'r_rmag'
cheung2007.drop(labels='Optical', axis=1, inplace=True)

cheung2007['Brightness1'] = cheung2007['rmag'].where(((cheung2007['rmag']>0) & (cheung2007['r_rmag']=='SDSS')), other=cheung2007['S1.4'])
cheung2007['Brightness2'] = cheung2007['Brightness1']
cheung2007['r SDSS'] = 'r SDSS'
cheung2007['Brightness_band1'] = cheung2007["r SDSS"].where(((cheung2007['r_rmag']=='SDSS') & (cheung2007['rmag']>0)), other='1.4 GHz NVSS')
cheung2007['Brightness_band2'] = cheung2007['Brightness2']
cheung2007.drop(labels='r SDSS', axis=1, inplace=True)

cheung2007['asinh model mag'] = 'asinh model mag'
cheung2007['Brightness_type1'] = cheung2007['asinh model mag'].where((cheung2007['r_rmag']=='SDSS'), other='Flux: mJy')
cheung2007['Brightness_type2'] = cheung2007['Brightness_type1']
cheung2007.drop(labels='asinh model mag', axis=1, inplace=True)

cheung2007['Sep'] = 0
cheung2007['Sep(kpc)'] = 0
cheung2007['delta_z'] = cheung2007['z1']-cheung2007['z2']
cheung2007['dV'] = (2.99e+5)*((1+cheung2007['z1'])**2 - (1+cheung2007['z2'])**2)/((1+cheung2007['z1'])**2+(1+cheung2007['z2'])**2)

# Adding in the info for Cheung's paper here
cheung2007['Paper(s)'] = "Cheung+2007"
cheung2007['BibCode(s)'] = "2007AJ....133.2097C"
cheung2007['DOI(s)'] = "https://doi.org/10.1086/513095"
cheung2007['Notes'] = " "

# Now dropping any irrelevant columns 
cheung2007.drop(labels=['[C2007]','rmag','r_rmag','z','r_z'], axis=1, inplace=True)

# Now rearranging the columns (and renaming if need be):

#cheung2007 = cheung2007[['System Type','Name1','Name2','Selection Method','Confirmation Method','z1', \
#                         'z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2',\
#                         'RA2_deg','Dec2_deg','Equinox','Coordinate_waveband','Coordinate_Source','Brightness1',\
#                         'Brightness_band1','Brightness_type1','Brightness2','Brightness_band2',\
#                         'Brightness_type2','Sep','Sep(kpc)','delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]

In [4]:
#print(cheung2007.columns)

In [5]:
#cheung2007

In [6]:
# Now reading in the information from table 1 of Cheung+2007, which was a literature compilation of X-shaped \
# radio sources

cheung2007_t1 = pd.read_csv('Tables/Cheung_2007/Cheung2007_table1.csv', sep=',')

cheung2007_t1['Name1'] = cheung2007_t1['System Name']
cheung2007_t1['Name2'] = cheung2007_t1['System Name']
cheung2007_t1['Selection Method'] = "X-Shaped Radio Source"
cheung2007_t1['Confirmation Method'] = -99
cheung2007_t1['z1'] = cheung2007_t1['Redshift']
cheung2007_t1['z1_type'] = -100
cheung2007_t1['z2'] = cheung2007_t1['Redshift']
cheung2007_t1['z2_type'] = -100

cheung2007_t1['RA1'] = cheung2007_t1['RA']
cheung2007_t1['Dec1']= cheung2007_t1['Dec']

cheung2007_t1['RA2'] = cheung2007_t1['RA']
cheung2007_t1['Dec2']= cheung2007_t1['Dec']

# Converting now the sexagesimal coordinates to degrees...
coordconvert = SkyCoord(ra = cheung2007_t1['RA1'], dec = cheung2007_t1['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
cheung2007_t1['RA1_deg'] = coordconvert.ra.degree
cheung2007_t1['Dec1_deg'] = coordconvert.dec.degree

cheung2007_t1['RA2_deg'] = cheung2007_t1['RA1_deg']
cheung2007_t1['Dec2_deg'] = cheung2007_t1['Dec1_deg']

cheung2007_t1['Brightness_band1'] = cheung2007_t1['Brightness_band1'].astype(str) + " (Simbad)"
cheung2007_t1['Brightness_type1'] = cheung2007_t1['Brightness_type1'].astype(str) + " (Check Simbad)"

cheung2007_t1['Brightness2'] = cheung2007_t1['Brightness1']
cheung2007_t1['Brightness_band2'] = cheung2007_t1['Brightness_band1']
cheung2007_t1['Brightness_type2'] = cheung2007_t1['Brightness_type1']

#cheung2007_t1['delta_z'] = cheung2007_t1['z1']-cheung2007_t1['z2']
cheung2007_t1['dV'] = (2.99e+5)*((1+cheung2007_t1['z1'])**2 - (1+cheung2007_t1['z2'])**2)/((1+cheung2007_t1['z1'])**2+(1+cheung2007_t1['z2'])**2)

cheung2007_t1['Coordinate_waveband'] = cheung2007_t1['Coordinate Waveband']
cheung2007_t1['Coordinate_Source'] = cheung2007_t1['Coordinate Source']

# Dropping unnecessary columns:
cheung2007_t1.drop(labels=['Component Name','J2000 Designation', 'System Name', 'Discovery Method', 'Redshift',\
                           'Redshift Type', 'RA', 'Dec'], axis=1, inplace=True)

# Tacking on Cheung+2007's paper info, just so I can note that Cheung+2007 lists all of these...
cheung2007_t1['Paper(s)'] = (cheung2007_t1['Paper(s)']).astype(str)+" ; Cheung+2007"
cheung2007_t1['BibCode(s)'] = cheung2007_t1['BibCode(s)'].astype(str)+" ; 2007AJ....133.2097C"
cheung2007_t1['DOI(s)'] = cheung2007_t1['DOI(s)'].astype(str)+" ; https://doi.org/10.1086/513095"
cheung2007_t1['Notes'] = " "


# Now rearranging the columns and renaming (if needed):

#cheung2007_t1 = cheung2007_t1[['System Type','Name1','Name2','Selection Method','Confirmation Method','z1',\
#                               'z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg',\
#                               'Dec2_deg','Equinox','Coordinate_waveband','Coordinate_Source','Brightness1',\
#                               'Brightness_band1','Brightness_type1','Brightness2','Brightness_band2',\
#                               'Brightness_type2','Sep(arcsec)','Sep(kpc)','delta_z','dV','Paper(s)',\
#                               'BibCode(s)','DOI(s)', 'Notes']]
# 
#cheung2007_t1.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1',\
#                               'z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg',\
#                               'Dec2_deg','Equinox','Coordinate_waveband','Coordinate_Source','Brightness1',\
#                               'Brightness_band1','Brightness_type1','Brightness2','Brightness_band2',\
#                               'Brightness_type2','Sep','Sep(kpc)','delta_z','dV','Paper(s)',\
#                               'BibCode(s)','DOI(s)', 'Notes']

In [7]:
#print(cheung2007_t1.columns)

In [8]:
# We need to add 3C 293 (Liu+2004) and for J0116 at RA and Dec 01 16 25.071 -47 22 40.67
# --> Liu+04 was added as an individual target.
# --> --> Need to come back to th eJ0116 bcause I don't remember what that is





In [9]:
# Now concatenating the two tables form Cheung+2007
frames=[cheung2007_t1,cheung2007]
cheung_master = pd.concat(frames).reset_index(drop=True)



In [10]:
#cheung_master

In [11]:
# Here we're loading in the X-shaped radio sources from Proctor+2011

proctor2011 = (Table.read('Tables/Proctor2011/table8.dat', readme = 'Tables/Proctor2011/ReadMe', format='ascii.cds')).to_pandas()

# There is some overlap between Proctor+2011 and Cheung+2007. Need to match those!


proctor2011['Name1'] = proctor2011['FCG']
proctor2011['Name2'] = proctor2011['FCG']
#proctor2011['z1'] = proctor2011['z']
#proctor2011['z2'] = proctor2011['z']
#proctor2011['z1_type'] = proctor2011['n_z']
#proctor2011['z2_type'] = proctor2011['n_z']

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(yang2019,proctor2011['Designation'])

proctor2011['RA1'] = proctor2011['RAh'].astype(str) + ':' + proctor2011['RAm'].astype(str) + ':' + proctor2011['RAs'].astype(str)
proctor2011['Dec1'] = proctor2011['DE-'].astype(str) + proctor2011['DEd'].astype(str) + ':' + proctor2011['DEm'].astype(str) + ':' + proctor2011['DEs'].astype(str)

# And now converting to get the coordinates in degrees rather than sexagesimal...
coordconvert = SkyCoord(ra = proctor2011['RA1'], dec = proctor2011['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

proctor2011['RA1_deg'] = coordconvert.ra.degree
proctor2011['Dec1_deg'] = coordconvert.dec.degree

#proctor2011['Coordinates'] = proctor2011['SDSSID']#.str.slice(start=1) # Stripping the J
#proctor2011['RA_test'] = proctor2011['Coordinates'].str.slice(start=1, stop=10) # Stripping the DEC parts 
#proctor2011['Dec_test'] = proctor2011['Coordinates'].str.slice(start=10, stop=19) # Stripping the RA parts
#proctor2011['RA'] = proctor2011['RA_test'].str.slice(start=0, stop=2)+":"+proctor2011['RA_test'].str.slice(start=2, stop=4)+":"+proctor2011['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#proctor2011['Dec'] = proctor2011['Dec_test'].str.slice(start=0, stop=3)+":"+proctor2011['Dec_test'].str.slice(start=3, stop=5)+":"+proctor2011['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#yang2019.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Adding in a second set of coordinates for the 'secondary'
proctor2011['RA2'] = proctor2011['RA1']
proctor2011['Dec2'] = proctor2011['Dec1']

proctor2011['RA2_deg'] = proctor2011['RA1_deg']
proctor2011['Dec2_deg'] = proctor2011['Dec1_deg']

# Adding details about the coordinates
proctor2011['Equinox'] = "J2000"
proctor2011['Coordinate_waveband'] = "Radio"
proctor2011['Coordinate_Source'] = "FIRST"

proctor2011['System Type'] = 'Recoil Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
proctor2011['Brightness1'] = -100
proctor2011['Brightness_band1'] = -100
proctor2011['Brightness_type1'] = -100

proctor2011['Brightness2'] = -100
proctor2011['Brightness_band2'] = -100
proctor2011['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#proctor2011['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#proctor2011['Sep(kpc)'] = proctor2011['Sep']*((cosmo.arcsec_per_kpc_proper(proctor2011['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#proctor2011['delta_z'] = proctor2011['z1']-proctor2011['z2']
#proctor2011['dV'] = (2.99e+5)*((1+proctor2011['z1'])**2 - (1+proctor2011['z2'])**2)/((1+proctor2011['z1'])**2+(1+proctor2011['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
proctor2011['Selection Method'] = "X-Shaped Radio Source" #DPSELs
proctor2011['Confirmation Method'] = "-99"
proctor2011['Paper(s)'] = "Proctor+2011"
proctor2011['BibCode(s)'] = "2011ApJS..194...31P"
proctor2011['DOI(s)'] = "https://doi.org/10.1088/0067-0049/194/2/31"

## And dropping any columns that we don't need....
#yang2019.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#yang2019 = proctor2011[['System Type','Name1','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#yang2019.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

# These two objects are flagged as having double-peaked emission lines
#J0818+1508 J1554+3811


#proctor2011



In [12]:
tunique, tmatches, idx1, idx2 = match_tables_fib(cheung_master,proctor2011,5)

# Adding the DOI, author, and bibcode info to all of the Smith+2010 rows here in the matches table...
for index, row in tmatches.iterrows():
    if row['Table_flag']!='Table2':
        tmatches.at[index, 'Paper(s)'] += ' ; Proctor+2011'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2011ApJS..194...31P' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0067-0049/194/2/31'

## Now clipping out all Smith+2010 rows from the matches table
tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)
#
## Concatenating everything together to generate a master table here
profx = pd.concat([tmatches,tunique]).sort_values(by='Name1').reset_index(drop=True)
profx.drop(labels=['index','Table_flag'], axis=1, inplace=True) #'level_0',

# There are 20 matches between these two tables
#print(len(tmatches))

# Verified that this works properly and proctor+ is attached to the correct number of objects in the final table


In [13]:
#profx

In [14]:
# Here we're adding in the information from Roberts+2018

roberts2018 = (Table.read('Tables/Roberts2018/table1.dat', readme = 'Tables/Roberts2018/ReadMe', format='ascii.cds')).to_pandas()
#Table 2 contains information on the VLA imaging
#Table 4 includes peak intensity information and figure tags

roberts_objs = roberts2018['[C2007]'].tolist()

for index, row in profx.iterrows():
    if row['Name1'] in roberts_objs:
        #print('True')
        profx.at[index, 'Paper(s)'] += ' ; Roberts+2018'
        profx.at[index, 'BibCode(s)'] += ' ; 2018ApJ...852...47R' 
        profx.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aa9c49'

# Verified that this matching process works
# Just go back and verify that the correct number of objects is actually matched
# profx

In [15]:
# Here we're adding in the information from Saripalli+2018
# Table 1 has redshift info, optical classes, XRG classification schemes, and size of radio extent
# Table 2 has information on hosts, emission lines, and radio power

# Here we're adding in the information from Roberts+2018

saripalli2018 = (Table.read('Tables/Saripalli2018/table1.dat', readme = 'Tables/Saripalli2018/ReadMe', format='ascii.cds')).to_pandas()
#Table 2 contains information on the VLA imaging
#Table 4 includes peak intensity information and figure tags

sar_objs = saripalli2018['[C2007]'].tolist()

for index, row in profx.iterrows():
    if row['Name1'] in sar_objs:
        #print('True')
        profx.at[index, 'Paper(s)'] += ' ; Saripalli+2018'
        profx.at[index, 'BibCode(s)'] += ' ; 2018ApJ...852...48S' 
        profx.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aa9c4b'

sarobjs = ['J0033−0149','J0049+0059','J0143−0119','J0821+2922','J1111+4050','J1128+1919','J1201−0703','J1210+1121','J1227+2155','J1330−0206','J1339−0016','J1522+4527','J2226+0125']      
for index, row in profx.iterrows():
    if row['Name1'] in sar_objs:
        #print('True')
        profx.at[index, 'Paper(s)'] += ' ; Saripalli+2018'
        profx.at[index, 'BibCode(s)'] += ' ; 2018ApJ...852...48S' 
        profx.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aa9c4b'
        profx.at[index, 'Notes'] += ' Saripalli+ reject this object for reasons listed in their Table 3.'
            
# This looks like it's all matched properly
# I've also verified that the correct number of objects int he final table are flagged having been examined by \
# Roberts+ and Saripalli+

# profx



In [16]:
# Now loading in the new X-shaped radio source from Yang+2019

# GO INTO YANG+ AND FLAG OBJECTS THEY QUOTE AS BEING DISCOVERED PREVIOUSLY!


yang2019t1 = (Table.read('Tables/Yang2019/table2.dat', readme = 'Tables/Yang2019/ReadMe', format='ascii.cds')).to_pandas()
yang2019t2 = (Table.read('Tables/Yang2019/table3.dat', readme = 'Tables/Yang2019/ReadMe', format='ascii.cds')).to_pandas()

yang2019t1['Name1'] = yang2019t1['SName']
yang2019t1['Name2'] = yang2019t1['SName']
yang2019t1['z1'] = yang2019t1['z']
yang2019t1['z2'] = yang2019t1['z']
yang2019t1['z1_type'] = yang2019t1['n_z']
yang2019t1['z2_type'] = yang2019t1['n_z']

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(yang2019,yang2019t1['Designation'])

yang2019t1['RA1'] = yang2019t1['RAh'].astype(str) + ':' + yang2019t1['RAm'].astype(str) + ':' + yang2019t1['RAs'].astype(str)
yang2019t1['Dec1'] = yang2019t1['DE-'].astype(str) + yang2019t1['DEd'].astype(str) + ':' + yang2019t1['DEm'].astype(str) + ':' + yang2019t1['DEs'].astype(str)

# And now converting to get the coordinates in degrees rather than sexagesimal...
coordconvert = SkyCoord(ra = yang2019t1['RA1'], dec = yang2019t1['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

yang2019t1['RA1_deg'] = coordconvert.ra.degree
yang2019t1['Dec1_deg'] = coordconvert.dec.degree

#yang2019t1['Coordinates'] = yang2019t1['SDSSID']#.str.slice(start=1) # Stripping the J
#yang2019t1['RA_test'] = yang2019t1['Coordinates'].str.slice(start=1, stop=10) # Stripping the DEC parts 
#yang2019t1['Dec_test'] = yang2019t1['Coordinates'].str.slice(start=10, stop=19) # Stripping the RA parts
#yang2019t1['RA'] = yang2019t1['RA_test'].str.slice(start=0, stop=2)+":"+yang2019t1['RA_test'].str.slice(start=2, stop=4)+":"+yang2019t1['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#yang2019t1['Dec'] = yang2019t1['Dec_test'].str.slice(start=0, stop=3)+":"+yang2019t1['Dec_test'].str.slice(start=3, stop=5)+":"+yang2019t1['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#yang2019.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Adding in a second set of coordinates for the 'secondary'
yang2019t1['RA2'] = yang2019t1['RA1']
yang2019t1['Dec2'] = yang2019t1['Dec1']

yang2019t1['RA2_deg'] = yang2019t1['RA1_deg']
yang2019t1['Dec2_deg'] = yang2019t1['Dec1_deg']

# Adding details about the coordinates
yang2019t1['Equinox'] = "J2000"
yang2019t1['Coordinate_waveband'] = "Optical"
yang2019t1['Coordinate_Source'] = "SDSS"

yang2019t1['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
yang2019t1['Brightness1'] = -100
yang2019t1['Brightness_band1'] = -100
yang2019t1['Brightness_type1'] = -100

yang2019t1['Brightness2'] = -100
yang2019t1['Brightness_band2'] = -100
yang2019t1['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#yang2019t1['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#yang2019t1['Sep(kpc)'] = yang2019t1['Sep']*((cosmo.arcsec_per_kpc_proper(yang2019t1['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#yang2019t1['delta_z'] = yang2019t1['z1']-yang2019t1['z2']
#yang2019t1['dV'] = (2.99e+5)*((1+yang2019t1['z1'])**2 - (1+yang2019t1['z2'])**2)/((1+yang2019t1['z1'])**2+(1+yang2019t1['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
yang2019t1['Selection Method'] = "Strong X-Shaped Radio Source" #DPSELs
yang2019t1['Confirmation Method'] = "-99"
yang2019t1['Paper(s)'] = "Yang+2019 ; Joshi+2019"
yang2019t1['BibCode(s)'] = "2019ApJS..245...17Y ; 2019ApJ...887..266J"
yang2019t1['DOI(s)'] = "https://doi.org/10.3847/1538-4365/ab4811 ; https://doi.org/ 10.3847/1538-4357/ab536f"
yang2019t1['Notes'] = " "


## And dropping any columns that we don't need....
#yang2019.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#yang2019 = yang2019t1[['System Type','Name1','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#yang2019.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

# These two objects are flagged as having double-peaked emission lines
#J0818+1508 J1554+3811

#yang2019t1



In [17]:
tunique, tmatches, idx1, idx2 = match_tables_fib(profx,yang2019t1,5)

# There are 12 matches between Yang+ and the main table

#tmatches
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in tmatches.iterrows():
    if row['Paper(s)']!='Yang+2019 ; Joshi+2019':
        tmatches.at[index, 'Paper(s)'] += ' ; Yang+2019 ; Joshi+2019'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2019ApJS..245...17Y ; 2019ApJ...887..266J' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4365/ab4811 ; https://doi.org/ 10.3847/1538-4357/ab536f'
        #tmatches.at[index, 'Notes'] += ''
        
# Now clipping out all Comerford+2013 rows from the matches table
tmatches = tmatches[tmatches['Paper(s)']!='Yang+2019 ; Joshi+2019'].reset_index(drop=True)

# Concatenating everything together to generate a master table here
profx = pd.concat([tmatches,tunique]).sort_values(by='Name1').reset_index(drop=True)
profx.drop(labels=['index'], axis=1, inplace=True)
#tunique

#profx

# This process appears to work properly


In [18]:
# Now loading in the new X-shaped radio source from Yang+2019
yang2019t2['Name1'] = yang2019t2['SName']
yang2019t2['Name2'] = yang2019t2['SName']
yang2019t2['z1'] = yang2019t2['z']
yang2019t2['z2'] = yang2019t2['z']
yang2019t2['z1_type'] = yang2019t2['n_z']
yang2019t2['z2_type'] = yang2019t2['n_z']

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(yang2019,yang2019t2['Designation'])

yang2019t2['RA1'] = yang2019t2['RAh'].astype(str) + ':' + yang2019t2['RAm'].astype(str) + ':' + yang2019t2['RAs'].astype(str)
yang2019t2['Dec1'] = yang2019t2['DE-'].astype(str) + yang2019t2['DEd'].astype(str) + ':' + yang2019t2['DEm'].astype(str) + ':' + yang2019t2['DEs'].astype(str)

# And now converting to get the coordinates in degrees rather than sexagesimal...
coordconvert = SkyCoord(ra = yang2019t2['RA1'], dec = yang2019t2['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

yang2019t2['RA1_deg'] = coordconvert.ra.degree
yang2019t2['Dec1_deg'] = coordconvert.dec.degree

#yang2019t2['Coordinates'] = yang2019t2['SDSSID']#.str.slice(start=1) # Stripping the J
#yang2019t2['RA_test'] = yang2019t2['Coordinates'].str.slice(start=1, stop=10) # Stripping the DEC parts 
#yang2019t2['Dec_test'] = yang2019t2['Coordinates'].str.slice(start=10, stop=19) # Stripping the RA parts
#yang2019t2['RA'] = yang2019t2['RA_test'].str.slice(start=0, stop=2)+":"+yang2019t2['RA_test'].str.slice(start=2, stop=4)+":"+yang2019t2['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#yang2019t2['Dec'] = yang2019t2['Dec_test'].str.slice(start=0, stop=3)+":"+yang2019t2['Dec_test'].str.slice(start=3, stop=5)+":"+yang2019t2['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#yang2019.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Adding in a second set of coordinates for the 'secondary'
yang2019t2['RA2'] = yang2019t2['RA1']
yang2019t2['Dec2'] = yang2019t2['Dec1']

yang2019t2['RA2_deg'] = yang2019t2['RA1_deg']
yang2019t2['Dec2_deg'] = yang2019t2['Dec1_deg']

# Adding details about the coordinates
yang2019t2['Equinox'] = "J2000"
yang2019t2['Coordinate_waveband'] = "Optical"
yang2019t2['Coordinate_Source'] = "SDSS"

yang2019t2['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
yang2019t2['Brightness1'] = -100
yang2019t2['Brightness_band1'] = -100
yang2019t2['Brightness_type1'] = -100

yang2019t2['Brightness2'] = -100
yang2019t2['Brightness_band2'] = -100
yang2019t2['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#yang2019t2['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#yang2019t2['Sep(kpc)'] = yang2019t2['Sep']*((cosmo.arcsec_per_kpc_proper(yang2019t2['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#yang2019t2['delta_z'] = yang2019t2['z1']-yang2019t2['z2']
#yang2019t2['dV'] = (2.99e+5)*((1+yang2019t2['z1'])**2 - (1+yang2019t2['z2'])**2)/((1+yang2019t2['z1'])**2+(1+yang2019t2['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
yang2019t2['Selection Method'] = "Probable X-Shaped Radio Source" #DPSELs
yang2019t2['Confirmation Method'] = "-99"
yang2019t2['Paper(s)'] = "Yang+2019"
yang2019t2['BibCode(s)'] = "2019ApJS..245...17Y"
yang2019t2['DOI(s)'] = "https://doi.org/10.3847/1538-4365/ab4811"
yang2019t2['Notes'] = " "


## And dropping any columns that we don't need....
#yang2019.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#yang2019 = yang2019t2[['System Type','Name1','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#yang2019.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

# this is flagged as having double-peaked emission lines:
# J1247+1948


#yang2019t2

In [19]:
tunique, tmatches, idx1, idx2 = match_tables_fib(profx,yang2019t2,5)

# There are 6 matches between the yang2019t2 table and the main table

#tmatches
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in tmatches.iterrows():
    if row['Paper(s)']!='Yang+2019':
        tmatches.at[index, 'Paper(s)'] += ' ; Yang+2019'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2019ApJS..245...17Y' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4365/ab4811'
        #tmatches.at[index, 'Notes'] += ''

# Now clipping out all Comerford+2013 rows from the matches table
tmatches = tmatches[tmatches['Paper(s)']!='Yang+2019'].reset_index(drop=True)

# Concatenating everything together to generate a master table here
profx = pd.concat([tmatches,tunique]).sort_values(by='Name1').reset_index(drop=True)
profx.drop(labels=['index'], axis=1, inplace=True)
#tunique

#profx

# This process appears to work properly
# I have examined the final output table, and we do indeed have 290 sources, as reported by Yang+.
# I have also confirmed that 106 objects were correclt flagged as being examined by Joshi+2019 as well.


In [20]:
#tmatches

In [21]:
# Adding in the bib information for the Lal+2019 targets, which were drawn from the cheung+07 list

lal2019 = ['J0113+0106','J0115-0000','J0702+5002','J0859-0433','J0914+1715','J0917+0523','J0924+4233','J1055-0707','J1130+0058','J1218+1955','J1309-0012','J1339-0016','J1406-0154','J1430+5217','J1600+2058','J1606+0000']

for index, row in profx.iterrows():
    if row['Name1'] in lal2019:
        print('True',row['Name1'])
        profx.at[index, 'Paper(s)'] += ' ; Lal+2019'
        profx.at[index, 'BibCode(s)'] += ' ; 2019AJ....157..195L' 
        profx.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-3881/ab1419'
        profx.at[index, 'Notes'] += ' Lal+2019 obtained additional radio imaging.'
            
#profx

# Verified that this matching process works and the correct number of objects list Lal+2019 in the final table.

# **************************
# EXCEPT: J1130+0058 COMES FROM WANG+2003 NOT FROM CHEUNG+2007, SO IT IS NOT IN THE MAIN TABLE PRIOR TO THIS
# We will need to manually include Wang+2003


True J0113+0106
True J0115-0000
True J0702+5002
True J0859-0433
True J0914+1715
True J0917+0523
True J0924+4233
True J1055-0707
True J1218+1955
True J1309-0012
True J1339-0016
True J1406-0154
True J1430+5217
True J1600+2058
True J1606+0000


In [22]:
# formatting the table here

profx['Sep'] = profx['Sep(arcsec)']

profx.drop(labels=['---','FCG','RAh','RAm','RAs','DE-','DEd','DEm','DEs','Size','u_Size','Type','RAc.h','RAc.m',\
                   'RAc.s','DEc.-','DEc.d','DEc.m','DEc.s','Group','Group2','Group3','Group4','Com','Table_flag',\
                   'SName','f_SName','z','n_z','rmag','RMag','S0.15','S5','r_S5','alph1','alph2',\
                   'Coordinate Waveband','Coordinate Source','S1.4','Sep(kpc)','Sep(arcsec)'],\
           axis=1, inplace=True)


In [23]:
# Now saving profx as a final table for output

profx.to_csv('Xshaped_radio_samples_DR1.csv', sep=',', index=False)



In [24]:
profx

,System Type,Equinox,Brightness1,Brightness_band1,Brightness_type1,Paper(s),BibCode(s),DOI(s),Notes,Name1,Name2,Selection Method,Confirmation Method,z1,z1_type,z2,z2_type,RA1,Dec1,RA2,Dec2,RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,Brightness2,Brightness_band2,Brightness_type2,dV,Coordinate_waveband,Coordinate_Source,Sep,delta_z
0,Recoil Candidate,J2000,17.00,B (Simbad),mag (Check Simbad),Spinrad+1985 ; Merritt+2002 ; Cheung+2007,; 2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,,3C 136.1,3C 136.1,X-Shaped Radio Source,-99,0.064000,-100.0,0.064000,-100.0,05:16:03.133,+24:58:25.47,05:16:03.133,+24:58:25.47,79.013054,24.973742,79.013054,24.973742,17.00,B (Simbad),mag (Check Simbad),0.0,Infrared,2MASS,0.0,NaN
1,Recoil Candidate,J2000,15.46,B (Simbad),mag (Check Simbad),Sandage+1966 ; Cheung+2007,nan ; 2007AJ....133.2097C,nan ; https://doi.org/10.1086/513095,,3C 192,3C 192,X-Shaped Radio Source,-99,0.059680,-100.0,0.059680,-100.0,08:05:35.0057673498,+24:09:50.334614040,08:05:35.0057673498,+24:09:50.334614040,121.395857,24.163982,121.395857,24.163982,15.46,B (Simbad),mag (Check Simbad),0.0,Optical,Gaia,0.0,NaN
2,Recoil Candidate,J2000,16.40,V (Simbad),mag (Check Simbad),Sandage+1966 ; Dennett-Thorpe+2002 ; Merritt+2...,; 2002MNRAS.330..609D ; 2002Sci...297.1310M ;...,; https://doi.org/10.1046/j.1365-8711.2002.05...,,3C 223.1,3C 223.1,X-Shaped Radio Source,-99,0.107500,-100.0,0.107500,-100.0,09:41:24.0272828657,+39:44:41.806341013,09:41:24.0272828657,+39:44:41.806341013,145.350114,39.744946,145.350114,39.744946,16.40,V (Simbad),mag (Check Simbad),0.0,Optical,Gaia,0.0,NaN
3,Recoil Candidate,J2000,16.30,B (Simbad),mag (Check Simbad),Schmidt+1965 ; Merritt+2002 ; Cheung+2007,; 2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,,3C 315,3C 315,X-Shaped Radio Source,-99,0.108300,-100.0,0.108300,-100.0,15:13:40.0227468596,+26:07:23.710955469,15:13:40.0227468596,+26:07:23.710955469,228.416761,26.123253,228.416761,26.123253,16.30,B (Simbad),mag (Check Simbad),0.0,Optical,Gaia,0.0,NaN
4,Recoil Candidate,J2000,18.00,V (Simbad),mag (Check Simbad),Smith+1976 ; Cheung+2007,nan ; 2007AJ....133.2097C,nan ; https://doi.org/10.1086/513095,,3C 379.1,3C 379.1,X-Shaped Radio Source,-99,0.256000,-100.0,0.256000,-100.0,18:24:33.061,+74:20:58.57,18:24:33.061,+74:20:58.57,276.137754,74.349603,276.137754,74.349603,18.00,V (Simbad),mag (Check Simbad),0.0,Infrared,2MASS,0.0,NaN
5,Recoil Candidate,J2000,16.50,V (Simbad),mag (Check Simbad),Sandage+1972 ; Dennett-Thorpe+2002 ; Merritt+2...,; 2002MNRAS.330..609D ; 2002Sci...297.1310M ;...,; https://doi.org/10.1046/j.1365-8711.2002.05...,,3C 403,3C 403,X-Shaped Radio Source,-99,0.059000,-100.0,0.059000,-100.0,19:52:15.7997953344,+02:30:24.235518851,19:52:15.7997953344,+02:30:24.235518851,298.065832,2.506732,298.065832,2.506732,16.50,V (Simbad),mag (Check Simbad),0.0,Optical,Gaia,0.0,NaN
6,Recoil Candidate,J2000,16.35,V (Simbad),mag (Check Simbad),Schmidt+1965 ; Merritt+2002 ; Cheung+2007,2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,,3C 433,3C 433,X-Shaped Radio Source,-99,0.101757,-100.0,0.101757,-100.0,21:23:44.582,+25:04:27.23,21:23:44.582,+25:04:27.23,320.935758,25.074231,320.935758,25.074231,16.35,V (Simbad),mag (Check Simbad),0.0,Infrared,NaN,0.0,NaN
7,Recoil Candidate,J2000,18.50,B (Simbad),mag (Check Simbad),Spinrad+1985 ; Merritt+2002 ; Cheung+2007,; 2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,,3C 52,3C 52,X-Shaped Radio Source,-99,0.290000,-100.0,0.290000,-100.0,01:48:28.884,+53:32:28.19,01:48:28.884,+53:32:28.19,27.120350,53.541164,27.120350,53.541164,18.50,B (Simbad),mag (Check Simbad),0.0,Infrared,2MASS,0.0,NaN
8,Recoil Candidate,J2000,18.50,B (Simbad),mag (Check Simbad),Smith+1980 ; Cheung+2009 ; Cheung+2007,nan ; 2007AJ....133.2097C,nan ; https://doi.org/10.1086/513095,,3C 63,3C 63,X-Shaped Radio Source,-99,0.175000,-100.0,0.175000,-100.